In [1]:
!pip install chainlit==1.0.400\
huggingface_hub==0.20.3\
langchain==0.1.11\
langchain_community==0.0.27\
langchain_core==0.1.30\
wikipedia \
pypdf \
llama-cpp-python \
chromadb \
pydantic==2.10.1 \
openai \
tiktoken \
python-dotenv \
pandas

In [2]:
# from torch import cuda, bfloat16
# import torch
# import transformers
# from transformers import AutoTokenizer
# from time import time
#import chromadb
#from chromadb.config import Settings

from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA 
from langchain.callbacks import StdOutCallbackHandler

# from huggingface_hub import hf_hub_download

import pandas as pd
import os
from constants import *
from dotenv import load_dotenv

# Loads environment variables from .env
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("Include your open api key in .env")

Include your open api key in .env


# Login to HuggingFace - Only for hub download but since we are using open ai this is not needded

In [3]:
# from huggingface_hub import notebook_login
# notebook_login()

# Scrape Wikipedia and store as langchain documents for chrommaDB

Use Scraper.ipynb, TODO convert to scraper.py


In [4]:
from langchain_core.documents import Document
wiki_df = pd.read_csv("scraped.csv")
wiki_df.head()

texts_from_wiki = wiki_df['content'].tolist()  

documents_from_wiki = [ Document(
    page_content= x,
    metadata={}
) for x in texts_from_wiki]


# Import User-Defined Documents

In [5]:
# Collate and gather text data from the PDFs in DATA_DIR
loader = PyPDFDirectoryLoader(DATA_DIR)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
docs = loader.load()
documents_from_user = text_splitter.split_documents(docs)


# Create Chroma DB -> We will merge all data sources (wikipedia, user-defined, etc.)

In [6]:
from langchain_core.documents import Document

# split text:
from langchain.llms import OpenAI
import os
from langchain.embeddings import OpenAIEmbeddings

# define openAI embeddings and LLMs
llm = OpenAI()
embeddings = OpenAIEmbeddings()

# Consolidate data:
all_documents = documents_from_wiki + documents_from_user


C:\Users\kai10\miniconda3\envs\chromerag\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
C:\Users\kai10\miniconda3\envs\chromerag\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
"""
    Create Chrome DB
"""

CHROMA_PATH = "chroma_db"
chroma_db = None
print("Creating Chroma Database from documents... This may take a while.")

# create ChromaDB from documents.
chroma_db = Chroma.from_documents(documents=all_documents, embedding=embeddings, persist_directory="./chroma_db")

Creating Chroma Database from documents... This may take a while.


# Run the actual chainlit / langchain combo

In [ ]:
import os
import argparse 
import json

os.system("chainlit run clrun.py")